####First Let's understand the basic Catalog + Volume Feature of Databricks
Filesystem Hierarchy of Volume in Databricks (DBFS)?  
Catalog -> /OurWorkspace/catalog/schema(database)/volume/folder/data files  
Tables Hierarchy of Databricks?  
Catalog -> /OurWorkspace/catalog/schema(database)/tables/data(dbfs filesystem/some other filesystems)

In [0]:
%sql
create catalog if not exists catalog1_dropme;
create database if not exists catalog1_dropme.schema1_dropme;
create volume if not exists catalog1_dropme.schema1_dropme.volume1_dropme;

In [0]:
dbutils.fs.mkdirs("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme")

If we need to create schema/volume/folder programatically, follow the below steps  
## Spark SQL  
###1.E(Extract)
L(Load)  
Inbuilt libraries sources/targets & Inbuilt data Formats  
2. Bread & Butter (T(Transformation) A(Analytical))

In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/wd36schema2/volume1/folder1")

## Learn How to Create Dataframes from filesystem using different options

Download the data from the below drive url  
[https://drive.google.com/drive/folders/1Tw7V9eBtUxy0xQMW38z3-bzWI_ewzLm6?usp=drive_link](url)

#### How Do We Write a Typical Spark Application (Core(Obsolete),SQL(Important),Streaming(Mid level important)) 
Before we Create Dataframe/RDD, what is the prerequisite? We need to create spark session object by instantiating sparksession class (by default databricks did that if you create a notebook)

In [0]:
from pyspark.sql.session import SparkSession
print(spark) #already instantiated by databricks
spark1 = SparkSession.builder.getOrCreate()
print(spark1) #we instantiated

Create a DBFS volume namely commondata and upload the above data in that volume What are other FS uri's available? file:///, hdfs:///, dbfs:///, gs:///, s3:///, adls:///, blob:///

#### How to Read/Extract the data from the filesytem and load it into the distributed memory for further processing/load - using diffent methodologies/options from different sources(fs & db) and different builtin formats (csv/json/orc/parquet/delta/tables)

In [0]:
#If I don't use any options in this csv function, what is the default functionality?
#1. By default it will consider ',' as a delimiter (sep='~')
#2. By default it will use _c0,_c1..._cn it will apply as column headers (header=True or toDF("","","") or we have more options to see further)
#3. By default it will treat all columns as string (inferSchema=True or we have more options to see further)
csv_df1=spark.read.csv("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_1")
#csv_df1.show(2)#display with produce output in a dataframe format
print(csv_df1.printSchema())
display(csv_df1)#display with produce output in a beautified table format, specific to databricks


In [0]:
#1. Header Concepts (Either we have define the column names or we have to use the column names from the data)
#Important option is...
#By default it will use _c0,_c1..._cn it will apply as column headers, but we are asking spark to take the first row as header and not as a data?
csv_df1=spark.read.csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_header",header=True)
print(csv_df1.printSchema())
csv_df1.show(2)
#csv_df1.write.csv("/Volumes/workspace/wd36schema2/volume1/folder1/outputdata")
#By default it will use _c0,_c1..._cn it will apply as column headers, if we use toDF(colnames) we can define our own headers..
csv_df2=spark.read.csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_1").toDF("id","fname","lname","age","prof")
#csv_df1.show(2)#display with produce output in a dataframe format
print(csv_df2.printSchema())
csv_df2.show(2)

In [0]:
#2. Printing Schema (equivalent to describe table)
csv_df1.printSchema()
csv_df2.printSchema()

In [0]:
#3. Inferring Schema 
# (Performance Consideration: Use this function causiously because it scans the entire data by immediately evaluating and executing
# hence, not good for large data or not good to use on the predefined schema dataset)
#sample data
#4004979,Tara,Drake,32,
#4004980,Earl,Hahn,34,Human resources assistant
#4004981,Don,Jones,THIRTY SIX,Lawyer
#csv_df1=spark.read.csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_2.txt",inferSchema=True).toDF("id","fname","lname","age","prof")
csv_df1=spark.read.csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_2",inferSchema=True).toDF("id","fname","lname","age","prof")
csv_df1.where("id in (4000001,4000002)").show(2)
csv_df1.printSchema()

In [0]:
#4. Using delimiter or seperator option
csv_df1=spark.read.csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_header_oth_del",header=True,sep='~')
csv_df1.show(2)
csv_df1.printSchema()

In [0]:
#5. Using different options to create dataframe with csv and other module... (2 methodologies with 3 ways of creating dataframes)
csv_df1=spark.read.csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_header_oth_del",inferSchema=True,header=True,sep='~')
csv_df1.show(2)
#or another way of creating dataframe (from any sources whether builtin or external)...
#option can be used for 1 or 2 option...
csv_df2=spark.read.option("header","True").option("inferSchema","true").option("sep","~").format("csv").load("dbfs:/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_header_oth_del")
csv_df2.show(2)
#options can be used for multiple options in one function as a parameter...
csv_df3=spark.read.options(header="True",inferSchema="true",sep="~").format("csv").load("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_header_oth_del")
csv_df3.show(2)
     

Generic way of read and load data into dataframe using fundamental options from built in sources (csv/orc/parquet/xml/json/table) (inferschema, header, sep)

In [0]:

csv_df1=spark.read.csv("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_header_oth_del",inferSchema=True,header=True,sep='~')
csv_df1.show(2)

Generic way of read and load data into dataframe using extended options from external sources (bigquery/redshift/athena/synapse) (tmpfolder, access controls)

In [0]:

#options can be used for multiple options in one function as a parameter...
csv_df3=spark.read.options(header="True",inferSchema="true",sep="~").format("csv").load("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust_other_del.txt")
csv_df3.show(2)

Reading data from multiple files

In [0]:

csv_df1=spark.read.csv("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust*",inferSchema=True,header=True,sep='~')
print(csv_df1.count())

In [0]:

csv_df1=spark.read.csv(path=["dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust*","/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_1"],inferSchema=True,header=True,sep=',')
print(csv_df1.count())


Requirement: I am getting data from different source systems of different regions (NY, TX, CA) into different landing pad (locations), how to access this data?

In [0]:
df_multiple_sources=spark.read.csv(path=["/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/sourcedata/NY","dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/sourcedata/TX"],inferSchema=True,header=True,sep=',',pathGlobFilter="custs_header_*",recursiveFileLookup=True)
#.toDF("cid","fn","ln","a","p")
print(df_multiple_sources.count())
df_multiple_sources.show(2)
     

Provide schema with SQL String or programatically (very very important)  
PySpark SQL Datatypes  
Data Types  
  
To provide schema (columname & datatype), what are the 2 basic options available that we learned so far ? inferSchema/toDF  
We are going to learn additionally 2 more options to handle schema (colname & datatype)?  
1. Using simple string format of define schema.  
IMPORTANT: 2. Using structure type to define schema.

In [0]:
#By default it will use _c0,_c1..._cn it will apply as column headers, if we use toDF(colnames) we can define our own headers..

csv_df1=spark.read.csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_1").toDF("id","fname","lname","age","prof")
print(csv_df1.printSchema())
csv_df1=spark.read.csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_1",inferSchema=True).toDF("id","fname","lname","age","prof")
print(csv_df1.printSchema())


In [0]:
#1. Using simple string format of define custom simple schema.
str_struct = "id integer, fname string, lname string, age string, prof string"
csv_df1=spark.read.schema(str_struct).csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_1")
print(csv_df1.printSchema())
csv_df1.show(2)

In [0]:
#2. Important part - /using structure type to define custom complex schema.
##4000001,Kristina,Chung,55,Pilot
#pattern - 
#import the types library based classes..
# define_structure = StructType([
#   StructField("colname", DataType(), True),
#   StructField("colname", DataType(), True)...])
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
custom_schema = StructType([StructField("id", IntegerType(), False), StructField("fname", StringType(), True), StructField("lname", StringType(), True), StructField("age",IntegerType(),True),StructField("prof",StringType())])
csv_df1=spark.read.schema(custom_schema).csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_header")
print(csv_df1.printSchema())
csv_df1.show(2)

5. Reading data from other formats (Try the below usecases after completing the 3-Basic-WriteOps)

In [0]:
df1 = spark.read.options(header="True",inferSchema="True",sep="~").csv("/Volumes/workspace/wd36schema/ingestion_volume/target/csvout/")
df1.show(2)
display(df1.take(2))

2. Reading json data

In [0]:

dfjson1=spark.read.json("/Volumes/workspace/wd36schema/ingestion_volume/target/jsonout")
dfjson1.show(2)
display(dfjson1.take(2))

3.Reading xml data

In [0]:
dfxml1=spark.read.xml("/Volumes/workspace/wd36schema/ingestion_volume/target/xmlout",rowTag="cust")
dfxml1.show(2)
display(dfxml1.take(3))

4. Reading Serialized data (orc/parquet/delta)

In [0]:
print("orcdata")
dforc1=spark.read.orc("/Volumes/workspace/wd36schema/ingestion_volume/target/orcout")
dforc1.show(2)
display(dforc1.take(2))
print("parquetdata")
dfparquet1=spark.read.parquet("/Volumes/workspace/wd36schema/ingestion_volume/target/parquetout")
dfparquet1.show(2)
display(dfparquet1.take(2))
print("deltadata")
dfdelta1=spark.read.format("delta").load("/Volumes/workspace/wd36schema/ingestion_volume/target/deltaout")
dfdelta1.show(2)
display(dfdelta1.take(2))

5. Reading delta (hive) table data (Lakehouse)

In [0]:
dftable1=spark.read.table("workspace.wd36schema.lh_usgtbl")
dftable1.show(2)
display(dftable1.take(2))

What are all the overall options we used in this notebook, for learning fundamental spark csv read operations?

1. How to create/manage Catalog & Volume
2. spark session
3. How to create some sample data and push it to the volume/folder/file
4. spark.read.csv operations & spark.read.format().load()
5. Few of the important read options under csv such as header, sep, inferSchema, toDF.
6. How to define custom schema/structure using mere string with colname & datatype or by using StructType([StructField(colname,DataType()...)]) (very important).
7. Few additional options such as samplingRatio, recursiveFileLookup & pathGlobFilter for accessing folders and subfolders with some pattern of filenames
8. How to Read data from different sources and different file formats